# Förberedelser
Vi laddar in data igen och splittar i tränings/test-dataset

In [ ]:
import numpy as np
import pandas as pd

# Läs datafilen
filename = './features_30_sec.csv'
df = pd.read_csv(filename, header = 0)


In [ ]:
# Visa de första 5 raderna
df.head()

In [ ]:
# välj kolumnen "filename"
df.filename

In [ ]:
# Välj några features och plocka ut labels
feature_names = ['chroma_stft_mean', 'rms_var', 'spectral_centroid_var']
features = df[feature_names]
labels = df.label

# En vanlig notering för features och labels är X och y. Vi kommer att använda det här.
X = features
y = labels

In [ ]:
# Splitta i train/test datasets

from sklearn.model_selection import train_test_split

# By default resulterar train_test_split i en (random) 75%/25%-split
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=42)

# SVN - Support Vector Machine
Som förra gången ska vi först göra fit() för att träna modellen på träningsdata, sedan predictar vi ny data.

För SVN ska vi dessutom införa ännu ett steg, skalning. 

Vi skalar alla variabler så att de är "lika stora" genom att skala dem baserat på sin standardavvikelse.
Detta är för att SVNs optimeringsmått, margin, är beroende av variablernas storlek.

In [ ]:
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame
from sklearn.svm import SVC

# StandardScaler kan du läsa mer om här
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
scaler = StandardScaler()
X_train_scaled_array = scaler.fit_transform(X_train)
X_test_scaled_array = scaler.fit_transform(X_test)

 # scaler ger en array av siffror. Vi vill ha data en dataframe för att det är trevligare senare (då har vi bland annat namn på kolumner)
X_train_scaled = DataFrame(X_train_scaled_array, columns=feature_names)
X_test_scaled = DataFrame(X_test_scaled_array, columns=feature_names)

#Initialisera en Support Vector Classifier med hyperparametern C = 1 och kernel = 'linear'
clf = SVC(C = 1.0, kernel='linear', random_state=42)
# Kör fit() på träningsdata för att träna modellen
clf.fit(X_train_scaled, y_train)

In [ ]:
# Nu kan vi se vad vi får för resultat av olika saker från vårt testdata
clf.predict(X_test_scaled[:10])

In [ ]:
# Och här är facit
y_test.iloc[:10]

In [ ]:
# Lite snyggare printat så kan man jämföra predictions med facit
print("- prediction -")
for element in clf.predict(X_test_scaled[:10]):
  print(element.ljust(10), end='')
print("\n- actual -")
for element in y_test.iloc[:10]:
  print(element.ljust(10), end='')

In [ ]:
# Nu kan vi beräkna accuracy (antal rätt/antal tester på hela test)

# Baseline är slumpad gissning: 1/10 = 0.1 eftersom det finns 10 klasser
# 'blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock'
clf.score(X_test_scaled, y_test)

## Egen övning (5-15 min)
Testa att ändra features och ändra parametrarna i SVN. Kolla vilka som finns här: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

Försök att slå 0.45 accurracy!!

# Grid search
I den egna övningen testade vi olika parametrar i SVN och det ledde till olika accurracy.

Det är en ganska etablerad sanning i ML att man måste testa sig fram lite tills man får bra resultat. Detta går att göra automatiskt med grid search.

Man gör ett "grid" med värden som datorn helt enkelt får söka igenom och sedan skriva en rapport automatiskt. Sedan kan man välja det bästa värdet, eller förfina sökningen där värdena såg ut att ge bra resultat.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame
from sklearn.svm import SVC

# Skala om data
scaler = StandardScaler()
X_train_scaled_array = scaler.fit_transform(X_train)
X_test_scaled_array = scaler.fit_transform(X_test)

X_train_scaled = DataFrame(X_train_scaled_array, columns=feature_names) # scaler ger en array av siffror. Vi vill ha en dataframe (så vi har namn på kolumner)
X_test_scaled = DataFrame(X_test_scaled_array, columns=feature_names)


parameters = {'kernel':('linear', 'rbf', 'poly', 'sigmoid'), 'C':[0.1, 1, 5, 10, 25, 100]}
search = GridSearchCV(SVC(), param_grid = parameters, n_jobs=2)
search.fit(X_train_scaled, y_train)

In [ ]:
search.cv_results_ # Printa en lista med resultat

In [ ]:
print(search.best_params_)
print(search.best_score_)

# Precision and recall
Nu ska vi göra två olika classifiers som optimerar för de olika värdena precision och recall.
1. En som sällan klassifierar icke-klassisk musik som klassisk musik.
2. En som oftast klassifierar klassisk musik rätt.

Men först ska vi se hur man får fram precision och recall i sklearn och som vanligt validerar vi på vårt testdata.

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# För att lära oss leker vi med ett nytt torrt teoretiskt exempel. Vi har en klassifierare som klassar mellan 1 och 0 samt lite ground truth-data

# y_true är det sanna värdet (ground truth)
y_true = [0, 1, 0, 0, 1, 1]
# y_pred är det värde vår modell estimerat (predicted)
y_pred = [0, 0, 1, 0, 0, 1]

# Lek runt lite med värdena ovan och få en känsla för hur precision och recall påverkas av y_true och y_pred.

# Precision och recall returnerar precision och recall per 
print('recall:', recall_score(y_true, y_pred))
print('precision:', precision_score(y_true, y_pred))

In [ ]:
# Nu ska vi använda det för utdatat från vår classifier ovan
y_our_data_true = y_test[:]
y_our_data_pred = clf.predict(X_test_scaled)

In [ ]:
# Eftersom precision och recall fungerar med en label åt gången, måste vi omvandla alla andra labels till samma label
y_our_data_true = [i if i == 'classical' else 'other' for i in y_our_data_true]
y_our_data_pred = [i if i == 'classical' else 'other' for i in y_our_data_pred]

In [ ]:
# Vi kan räkna ut precision och recall manuellt
num_true_positive = 0
num_true = 0
num_predicted = 0
for i in range(len(y_our_data_true)):
    if y_our_data_true[i] == 'classical' and y_our_data_pred[i] == 'classical':
        num_true_positive += 1
    if y_our_data_true[i] == 'classical':
        num_true += 1
    if y_our_data_pred[i] == 'classical':
        num_predicted += 1
print('recall', num_true_positive/num_true)
print('precision', num_true_positive/num_predicted)
    

In [ ]:
# Eller använda inbyggda metrics från sklearn (notera pos_label='classical'. Det indikerar vilken som anses vara "positiv" träff)
print('recall:', recall_score(y_our_data_true, y_our_data_pred, pos_label='classical'))
print('precision:', precision_score(y_our_data_true, y_our_data_pred, pos_label='classical'))

In [ ]:
# Ett annat sätt att få fram alla metrics på en gång (sorry att jag tog en svårare väg först men hoppas det var instruktivt)
# använd classification_report
from sklearn.metrics import classification_report
print(classification_report(y_our_data_true, y_our_data_pred))

In [ ]:
# Då kan vi få en sammanfattning för ALLA labels precision och recall-scores
print(classification_report(y_test[:], clf.predict(X_test_scaled)))

# Egen övning


- Det är ganska lätt att göra en classifier som har perfekt precision *eller* perfekt recall. Hur? (Kom på det själv!) Har du lust kan du koda en sådan classifier. Antingen själv eller använd sklearns classifier och ändra parametrar.
- Läs på om f1-score. Det är en kombination mellan precision och recall som kan användas när man jämför classifiers som är bra på de olika sakerna.
- Testa andra classifiers från sklearn https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html
- Gör en så bra classifier du kan. Använd fler features, ändra hyperparametrar och testa andra classifiers från sklearn. Gör två olika som är bra på dessa olika saker 
  - a. Maximera hur bra den är på att klassifiera en särskilt genre (f1-score för den genren)
  - b. Maximera hur bra den är på att klassifiera alla genres
- Varför går det (oftast) inte att designa en perfekt classifier? Filosofera! Ta med dina tankar till imorgon :)